In [1]:
import pandas as pd
from SwingTrading import SwingTrading

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)


In [2]:
#Create a SwingTrading object
swing_trading = SwingTrading('AAPL', '2021-01-01', '2022-12-31', '1d', 0.05, 0.03, 5)
# Retrieve the data
data = swing_trading.get_data()
data = swing_trading.add_rolling_columns(5)

[*********************100%***********************]  1 of 1 completed


In [3]:
data = swing_trading.add_moving_averages()
data = swing_trading.add_bollinger_bands()
data = swing_trading.add_macd()
data = swing_trading.add_rsi()
data = swing_trading.add_vwap()
data = swing_trading.add_Accumulation_Distribution()
data = swing_trading.add_volatility()
data = swing_trading.add_accumulation_distribution()
data = swing_trading.add_on_balance_volume()
data = swing_trading.add_stochastic_oscillator()
data = swing_trading.add_williams()
data = swing_trading.add_cci()
data = swing_trading.add_fibonacci()
data = swing_trading.add_label()
data = swing_trading.add_momentum()
data = swing_trading.add_aroon()
data = swing_trading.add_chaikin_oscillator()
data = swing_trading.add_trend_line()
data = swing_trading.add_Volume_Profile_Analysis()
data = swing_trading.add_Candlestick_Pattern_Analysis()
data = swing_trading.add_momentum_indicators()
data = swing_trading.add_rsi_divergence()
data = swing_trading.add_parabolic_sar()
data = swing_trading.add_ADX()
data = swing_trading.add_ichimoku_cloud()
data = swing_trading.add_OBV()
data = swing_trading.add_bollinger_band_width()
data = swing_trading.add_ichimoku_kinko_hyo()
data = swing_trading.add_elliott_wave_analysis()
data = swing_trading.add_moving_average_convergence_divergence()
data = swing_trading.add_on_balance_volume()
data = swing_trading.add_stochRSI()
data = swing_trading.add_keltner_channels()
data = swing_trading.add_gann_fan()

data = swing_trading.add_triple_exponential_average()
data = swing_trading.add_rate_of_change()
data = swing_trading.add_detrended_price_oscillator()

data = swing_trading.add_mass_index()
data = swing_trading.add_vortex_indicator()
data = swing_trading.add_kst_oscillator()
data = swing_trading.add_chaikin_money_flow()
data = swing_trading.add_average_directional_index()
data = swing_trading.add_ultimate_oscillator()
data = swing_trading.add_linear_regression_indicator()
data = swing_trading.add_negative_volume_index()
data = swing_trading.add_mcclellan_oscillator()
data = swing_trading.add_kaufman_adaptive_moving_average()
data = swing_trading.add_moving_average_of_oscillator()
data = swing_trading.add_klinger_volume_oscillator()


data = swing_trading.add_coppock_curve()
data = swing_trading.add_elder_ray_index()
data = swing_trading.add_force_index()
data = swing_trading.add_elder_impulse_system()
data = swing_trading.add_chandelier_exit()
data = swing_trading.add_keltner_channel_breakout_system()
data = swing_trading.add_parabolic_sar_adx_combo()
data = swing_trading.add_dmi_stochastic_system()
data = swing_trading.add_swing_index()
data = swing_trading.add_andrews_pitchfork()
data = swing_trading.add_ichimoku_cloud()
data = swing_trading.add_relative_strength_index_wilder_smoothing()
data = swing_trading.add_rainbow_charts()
data = swing_trading.add_market_profile()
data = swing_trading.add_elder_safe_zone_strategy()
data = swing_trading.add_moving_average_envelope()
data = swing_trading.add_bollinger_band_squeeze()
data = swing_trading.add_obvios_momentum_indicator()
data = swing_trading.add_keltner_channel_breakout_system()
data = swing_trading.add_money_flow_index()
data = swing_trading.add_donchian_channel_breakout_system()
data = swing_trading.add_average_directional_movement_index()
data = swing_trading.add_commodity_channel_index()
data = swing_trading.add_stochastic_rsi()
data = swing_trading.add_volume_weighted_average_price()
data = swing_trading.add_connors_rsi()
data = swing_trading.add_super_trend_indicator()
data = swing_trading.add_average_true_range_stop()
data = swing_trading.add_swing_index()
data = swing_trading.add_ichimoku_kinko_hyo()
data = swing_trading.add_keltner_channel_volatility_breakout_system()
data = swing_trading.add_triple_screen_trading_system()
data = swing_trading.add_standard_deviation_channel()
data = swing_trading.add_adaptive_moving_average()
data = swing_trading.add_chande_momentum_oscillator()
data = swing_trading.add_volatility_stop()
data = swing_trading.add_volume_zone_oscillator()
data = swing_trading.add_vama()
data = swing_trading.add_acceleration_deceleration_oscillator()
data = swing_trading.add_directional_movement_index()
data = swing_trading.add_elder_ray_bull_bear_power()
data = swing_trading.add_detrended_price_oscillator()
data = swing_trading.add_trix_indicator()


In [4]:
data.dropna(inplace=True)

int64_cols = [col for col in data.select_dtypes(include='int64').columns if data[col].max() > 1 or data[col].min() < 0]
float64_cols = [col for col in data.select_dtypes(include='float64').columns if data[col].max() > 1 or data[col].min() < 0]
object_cols =  data.select_dtypes(include='object').columns
date_cols = data.select_dtypes(include='datetime64[ns]').columns

# scale cols to between -1 and 1
scale_cols = float64_cols + int64_cols
for col in scale_cols:
    data = swing_trading.scale_column(col,50)

In [12]:
data['high_close_change'] = data['High_5'] / data['Close'] - 1
data['low_close_change'] = data['Low_5'] / data['Close'] - 1

In [13]:
import numpy as np
data['lable'] = np.where((data['high_close_change'
                              ] >.035) & (data['High_forward_5'] < data['Low_forward_5']), 'Yes - First',
                        np.where((data['high_close_change'] >.035), 'Yes'), 
                          np.where((data['low_close_change'] < -0.025) & (data['High_forward_5'
                                                                            ] < data['Low_forward_5']), 'No - First',
                                   np.where((data['low_close_change'] < -0.025), 'No'), 'Flat'
                                   
                        ))

ValueError: either both or neither of x and y should be given

In [ ]:
self.data[rows_forward_var] = np.where(self.data[max_lag_var] > self.data[min_lag_var], 'UpTrend', 
                                            np.where(self.data[min_lag_var] > self.data[max_lag_var], 'DownTrend','NoTrend'))

       

In [8]:
data.tail(10)

,Date,Open,High,Low,Close,Adj Close,Volume,High_5,High_forward_5,Low_5,Low_forward_5,High_Low_forward_5,high_close_change,low_close_change
493,2022-12-16,136.690002,137.649994,133.729996,134.509995,134.509995,160156900,136.809998,3.0,129.639999,5.0,DownTrend,0.017099,-0.036205
494,2022-12-19,135.110001,135.199997,131.320007,132.369995,132.369995,79592600,136.809998,2.0,128.720001,5.0,DownTrend,0.033542,-0.027574
495,2022-12-20,131.389999,133.250000,129.889999,132.300003,132.300003,77432800,136.809998,1.0,125.870003,5.0,DownTrend,0.034089,-0.048602
496,2022-12-21,132.979996,136.809998,132.750000,135.449997,135.449997,85928000,134.559998,1.0,125.870003,4.0,DownTrend,-0.006571,-0.070727
497,2022-12-22,134.350006,134.559998,130.300003,132.229996,132.229996,77852100,132.419998,1.0,125.870003,3.0,DownTrend,0.001437,-0.048098
498,2022-12-23,130.919998,132.419998,129.639999,131.860001,131.860001,63814900,NaN,NaN,NaN,NaN,NoTrend,NaN,NaN
499,2022-12-27,131.380005,131.410004,128.720001,130.029999,130.029999,69007800,NaN,NaN,NaN,NaN,NoTrend,NaN,NaN
500,2022-12-28,129.669998,131.029999,125.870003,126.040001,126.040001,85438400,NaN,NaN,NaN,NaN,NoTrend,NaN,NaN
501,2022-12-29,127.989998,130.479996,127.730003,129.610001,129.610001,75703700,NaN,NaN,NaN,NaN,NoTrend,NaN,NaN
502,2022-12-30,128.410004,129.949997,127.430000,129.929993,129.929993,76960600,NaN,NaN,NaN,NaN,NoTrend,NaN,NaN


In [ ]:
136

In [ ]:


def swing_trading_label(OHLC_obs, up_percent, down_percent):
  label = 0
  
  current_high = OHLC_obs[0]
  current_low = OHLC_obs[0]
  
  for price in OHLC_obs[1:]:
    current_high = max(current_high, price)
    current_low = min(current_low, price)
  
  if (price - current_low) / current_low >= down_percent / 100 and (current_high - price) / current_high >= up_percent / 100:
    label = 1
    
  return label up_percent, down_percent):
    # Initializing label
    label = 0
    
    # Looping over OHLC observations
    for i in range(num_OHLC):
        # Checking if stock has moved up by at least up_percent
        if i > 0 and OHLC[i] - OHLC[i-1] >= OHLC[i-1] * up_percent:
            # Checking if stock has not moved down by more than down_percent
            if i > 0 and OHLC[i] - OHLC[i-1] <= OHLC[i-1] * down_percent:
                # Setting label to 1
                label = 1
    
    # Returning label
    return label

In [6]:
data.tail()

,Date,High_Low_forward_5,Label,Bullish_Engulfing,Bearish_Engulfing,Hammer,Shooting_Star,Doji,Harami,Hanging_Man,Inverted_Hammer,Tweezer_Bottom,Tweezer_Top,Bullish_Three_Line_Strike,Bearish_Three_Line_Strike,Bullish_Abandoned_Baby,Bearish_Abandoned_Baby,Bullish_Kicker,Bearish_Kicker,Bullish_Kick_Backs,Bearish_Kick_Backs,Bullish_Harami_Cross,Bearish_Harami_Cross,Bullish_Engulfing_Pattern,Bearish_Engulfing_Pattern,Mass_Index_9,Mass_Index_25,Average_Directional_Index,Stochastic_RSI,scaled_50_Open,scaled_50_High,scaled_50_Low,scaled_50_Close,scaled_50_Adj Close,scaled_50_High_5,scaled_50_High_forward_5,scaled_50_Low_5,scaled_50_Low_forward_5,scaled_50_MA_5,scaled_50_MA_10,scaled_50_MA_20,scaled_50_MA_50,scaled_50_MA_100,scaled_50_MA_200,scaled_50_Mid_Bollinger_Band,scaled_50_Upper_Bollinger_Band,scaled_50_Lower_Bollinger_Band,scaled_50_EMA_12,scaled_50_EMA_26,scaled_50_MACD,scaled_50_Signal_Line,scaled_50_Change,scaled_50_Gain,scaled_50_Loss,scaled_50_Avg_Gain_14,scaled_50_Avg_Loss_14,scaled_50_RS,scaled_50_RSI,scaled_50_VWAP,scaled_50_Accumulation_Distribution,scaled_50_Vol_5,scaled_50_Vol_10,scaled_50_Vol_20,scaled_50_Vol_50,scaled_50_Vol_100,scaled_50_Vol_200,scaled_50_Money_Flow_Multiplier,scaled_50_Money_Flow_Volume,scaled_50_ADL,scaled_50_Stochastic_Oscillator_K,scaled_50_Stochastic_Oscillator_D,scaled_50_Williams_R,scaled_50_TP,scaled_50_CCI,scaled_50_Fibonacci_0.236,scaled_50_Fibonacci_0.382,scaled_50_Fibonacci_0.50,scaled_50_Fibonacci_0.618,scaled_50_Fibonacci_1.00,scaled_50_Fibonacci_1.27,scaled_50_Fibonacci_1.618,scaled_50_Momentum_10,scaled_50_Aroon_Up,scaled_50_Aroon_Down,scaled_50_ADL_10,scaled_50_Chaikin_Osc,scaled_50_Trend_Line,scaled_50_Volume_Profile_Analysis,scaled_50_Momentum_12,scaled_50_Momentum_26,scaled_50_RSI_Divergence_12,scaled_50_RSI_Divergence_26,scaled_50_Parabolic_SAR_12,scaled_50_Parabolic_SAR_26,scaled_50_PDM,scaled_50_MDM,scaled_50_TR,scaled_50_PDI,scaled_50_MDI,scaled_50_ADX,scaled_50_Tenkan_Sen,scaled_50_Kijun_Sen,scaled_50_Senkou_Span_A,scaled_50_Senkou_Span_B,scaled_50_Chikou_Span,scaled_50_OBV,scaled_50_Bollinger_Band_Width_12,scaled_50_Bollinger_Band_Width_26,scaled_50_Ichimoku_Kinko_Hyo_Tenkan_Sen,scaled_50_Ichimoku_Kinko_Hyo_Kijun_Sen,scaled_50_Ichimoku_Kinko_Hyo_Senkou_Span_A,scaled_50_Ichimoku_Kinko_Hyo_Senkou_Span_B,scaled_50_Ichimoku_Kinko_Hyo_Chikou_Span,scaled_50_Impulse_Waves,scaled_50_Corrective_Waves,scaled_50_Extension_Waves,scaled_50_Corrective_Extension_Waves,scaled_50_MACD_12_26,scaled_50_MACD_Signal_9,scaled_50_MACD_Histogram,scaled_50_StochRSI_K,scaled_50_StochRSI_D,scaled_50_Keltner_Channel_Mid,scaled_50_Keltner_Channel_Upper,scaled_50_Keltner_Channel_Lower,scaled_50_Gann_Fan_1,scaled_50_Gann_Fan_2,scaled_50_Gann_Fan_3,scaled_50_Gann_Fan_4,scaled_50_Gann_Fan_5,scaled_50_Gann_Fan_6,scaled_50_Gann_Fan_7,scaled_50_Triple_EMA_5,scaled_50_Triple_EMA_10,scaled_50_Triple_EMA_20,scaled_50_Triple_EMA_50,scaled_50_Triple_EMA_100,scaled_50_Triple_EMA_200,scaled_50_Rate_of_Change_5,scaled_50_Rate_of_Change_10,scaled_50_Rate_of_Change_20,scaled_50_Rate_of_Change_50,scaled_50_Rate_of_Change_100,scaled_50_Rate_of_Change_200,scaled_50_Detrended_Price_Oscillator,scaled_50_VI_Positive,scaled_50_VI_Negative,scaled_50_Vortex_Indicator,scaled_50_KST_10,scaled_50_KST_15,scaled_50_KST_20,scaled_50_KST_30,scaled_50_KST_50,scaled_50_KST_Oscillator,scaled_50_Chaikin_Money_Flow,scaled_50_High_Minus_Low,scaled_50_High_Minus_Close_Previous,scaled_50_Close_Previous_Minus_Low,scaled_50_True_Range,scaled_50_DM_Plus,scaled_50_DM_Minus,scaled_50_DI_Plus,scaled_50_DI_Minus,scaled_50_Total_TR,scaled_50_BP,scaled_50_Average_7_TR,scaled_50_Average_14_TR,scaled_50_Average_28_TR,scaled_50_Raw_UO,scaled_50_Average_7_BP,scaled_50_Average_14_BP,scaled_50_Average_28_BP,scaled_50_Ultimate_Oscillator,scaled_50_Linear_Regression_Slope,scaled_50_Linear_Regression_Intercept,scaled_50_Linear_Regression_Indicator,scaled_50_McClellan_Oscillator,scaled_50_KAMA,scaled_50_MACD_Signal,scaled_50_KVO_High,scaled_50_KVO_Low,scaled_5

In [21]:
data.dropna(inplace=True)
int64_cols = data.select_dtypes(include='int64')
float64_cols = data.select_dtypes(include='float64')
object_cols =  data.select_dtypes(include='object')
date_cols = data.select_dtypes(include='datetime64[ns]')

In [5]:
data.isna().sum()

Date                                    0
Open                                    0
High                                    0
Low                                     0
Close                                   0
Adj Close                               0
Volume                                  0
High_5                                  5
High_forward_5                          5
Low_5                                   5
Low_forward_5                           5
High_Low_forward_5                      0
MA_5                                    4
MA_10                                   9
MA_20                                  19
MA_50                                  49
MA_100                                 99
MA_200                                199
Mid_Bollinger_Band                     19
Upper_Bollinger_Band                   19
Lower_Bollinger_Band                   19
EMA_12                                  0
EMA_26                                  0
MACD                              

In [7]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume,High_5,High_forward_5,Low_5,Low_forward_5,MA_5,MA_10,MA_20,MA_50,MA_100,MA_200,Mid_Bollinger_Band,Upper_Bollinger_Band,Lower_Bollinger_Band,EMA_12,EMA_26,MACD,Signal_Line,Change,Gain,Loss,Avg_Gain_14,Avg_Loss_14,RS,RSI,VWAP,Accumulation_Distribution,Vol_5,Vol_10,Vol_20,Vol_50,Vol_100,Vol_200,Money_Flow_Multiplier,Money_Flow_Volume,ADL,OBV_Change,On-Balance_Volume,Stochastic_Oscillator_K,Stochastic_Oscillator_D,Williams_R,TP,CCI,Fibonacci_0.236,Fibonacci_0.382,Fibonacci_0.50,Fibonacci_0.618,Fibonacci_1.00,Fibonacci_1.27,Fibonacci_1.618,Momentum_10,Aroon_Up,Aroon_Down,ADL_10,Chaikin_Osc,Trend_Line,Volume_Profile_Analysis,Bullish_Engulfing,Bearish_Engulfing,Hammer,Shooting_Star,Doji,Harami,Hanging_Man,Inverted_Hammer,Tweezer_Bottom,Tweezer_Top,Bullish_Three_Line_Strike,Bearish_Three_Line_Strike,Bullish_Abandoned_Baby,Bearish_Abandoned_Baby,Bullish_Kicker,Bearish_Kicker,Bullish_Kick_Backs,Bearish_Kick_Backs,Bullish_Harami_Cross,Bearish_Harami_Cross,Bullish_Engulfing_Pattern,Bearish_Engulfing_Pattern,Momentum_12,Momentum_26,RSI_Divergence_12,RSI_Divergence_26,Parabolic_SAR_12,Parabolic_SAR_26,PDM,MDM,TR,PDI,MDI,ADX,Tenkan_Sen,Kijun_Sen,Senkou_Span_A,Senkou_Span_B,Chikou_Span,OBV,Bollinger_Band_Width_12,Bollinger_Band_Width_26,Ichimoku_Kinko_Hyo_Tenkan_Sen,Ichimoku_Kinko_Hyo_Kijun_Sen,Ichimoku_Kinko_Hyo_Senkou_Span_A,Ichimoku_Kinko_Hyo_Senkou_Span_B,Ichimoku_Kinko_Hyo_Chikou_Span,Impulse_Waves,Corrective_Waves,Extension_Waves,Corrective_Extension_Waves,MACD_12_26,MACD_Signal_9,MACD_Histogram,StochRSI_K,StochRSI_D,Keltner_Channel_Mid,Keltner_Channel_Upper,Keltner_Channel_Lower,Gann_Fan_1,Gann_Fan_2,Gann_Fan_3,Gann_Fan_4,Gann_Fan_5,Gann_Fan_6,Gann_Fan_7,Triple_EMA_5,Triple_EMA_10,Triple_EMA_20,Triple_EMA_50,Triple_EMA_100,Triple_EMA_200,Rate_of_Change_5,Rate_of_Change_10,Rate_of_Change_20,Rate_of_Change_50,Rate_of_Change_100,Rate_of_Change_200,Detrended_Price_Oscillator,Mass_Index_9,Mass_Index_25,VI_Positive,VI_Negative,Vortex_Indicator,KST_10,KST_15,KST_20,KST_30,KST_50,KST_Oscillator,Chaikin_Money_Flow,High_Minus_Low,High_Minus_Close_Previous,Close_Previous_Minus_Low,True_Range,DM_Plus,DM_Minus,DI_Plus,DI_Minus,Average_Directional_Index,Total_TR,BP,Average_7_TR,Average_14_TR,Average_28_TR,Raw_UO,Average_7_BP,Average_14_BP,Average_28_BP,Ultimate_Oscillator,Linear_Regression_Slope,Linear_Regression_Intercept,Linear_Regression_Indicator,Negative_Volume_Index,McClellan_Oscillator,KAMA,MACD_Signal,KVO_High,KVO_Low,Klinger_Volume_Oscillator,Coppock_Curve,Bull_Power,Bear_Power,Force_Index,Force_Index_EMA,12-period EMA,26-period EMA,Signal,MACD_Histogram_EMA,MACD_Histogram_EMA_2,ATR,ATR_EMA,Chandelier_Exit,Keltner_Channel_Upper_Band,Keltner_Channel_Lower_Band,Parabolic_SAR,High_Minus_Previous_Close,Low_Minus_Previous_Close,DMI_Plus,DMI_Minus,DMI_Plus_MA,DMI_Minus_MA,DMI_Diff,DMI_Diff_MA,Swing_Index,Andrews_Pitchfork_Line_1,Andrews_Pitchfork_Line_2,Andrews_Pitchfork_Line_3,Andrews_Pitchfork_Line_1_MA,Andrews_Pitchfork_Line_2_MA,Andrews_Pitchfork_Line_3_MA,Avg_Gain,Avg_Loss,RSI_5,RSI_14,RSI_21,RSI_28,RSI_35,High_Volume,Low_Volume,High_Volume_Average,Low_Volume_Average,EMA_13,EMA_34,Histogram,Safe_Zone,MA_20_Upper,MA_20_Lower,MA_50_Upper,MA_50_Lower,MA_200_Upper,MA_200_Lower,BB_Upper,BB_Lower,BB_Squeeze,OBVios_Momentum,Money_Flow_Index,Donchian_Channel_Upper_Band,Donchian_Channel_Lower_Band,Average_Directional_Movement_Index,Commodity_Channel_Index,Stochastic_RSI,Volume_Weighted_Average_Price,Connors_RSI,Super_Trend,ATR_Stop,KC_Upper_Band,KC_Lower_Band,KC_Mid_Band,TS_First_Screen,TS_Second_Screen,TS_Third_Screen,SDC_Upper_Band,SDC_Lower_Band,SDC_Mid_Band,Adaptive_MA,CMO,Volatility_Stop,Volume_Zone_Oscillator,VAMA_5,VAMA_10,VAMA_20,VAMA_50,VAMA_100,VAMA_200,AccDecel_Oscillator_5,AccDecel_Oscillator_10,AccDecel_Oscillator_20,AccDecel_Oscillator_50,AccDecel_Oscillator_100,AccDecel_Oscillator_200,DI_Plus_EMA,DI_Minus_EMA,DI_Diff,DI_Sum,DX,Bull_Power_EMA,Bear_Power_EMA,TRIX_5,TRIX_10,TRIX_20,TRIX_50,TRIX_100,TR

In [ ]:
scale_column

In [23]:
scale_column(data, 'High')[['High','scaled_High']].tail(6)


0             NaN
1             NaN
2      133.610001
3      131.740005
4      131.630005
          ...    
498    136.809998
499    134.559998
500    132.419998
501    131.410004
502    131.029999
Name: High, Length: 503, dtype: float64 0             NaN
1             NaN
2      131.740005
3      131.050003
4      131.050003
          ...    
498    134.559998
499    132.419998
500    131.410004
501    131.029999
502    130.479996
Name: High, Length: 503, dtype: float64


,High,scaled_High
497,134.559998,0.367977
498,132.419998,-0.951111
499,131.410004,-0.471960
500,131.029999,-0.376245
501,130.479996,-1.447358
502,129.949997,-0.963629


In [24]:
(129.949997-131.029999)/(131.029999-130.479996)

-1.9636292892947858

In [21]:
def scale_column(self, col_name, period):
    """
    Scales a given column of a pandas DataFrame between 0 and 1 using the 
    columns prior 20 high and low values.

    Parameters
    ----------
    data : DataFrame
        The pandas DataFrame containing the column to be scaled
    col_name : string
        The name of the column to be scaled

    Returns
    -------
    DataFrame
        A pandas DataFrame with the scaled column
    """
    
    high = self.data[col_name].rolling(period).max().shift(1)
    low = self.data[col_name].rolling(period).min().shift(1)
    self.data['scaled_' + col_name] = (self.data[col_name] - low)/(high - low)
    del self.data[col_name]
    return self.data

In [14]:
high_5 = list(data['High'].rolling(5).max())[5:]
max_high_lag = list(data['High'].rolling(5).apply(lambda x: list(x).index(max(x))+1))[5:]

empty_ = [0]*5

high_5 = high_5 + empty_
max_high_lag = max_high_lag + empty_

data['high_5'] = high_5
data['max_high_lag'] = max_high_lag


In [46]:
def add_rolling_columns(column_name, rows_forward):
    rolling_max = list(data[column_name].rolling(rows_forward).max())[rows_forward:]
    max_lag = list(data[column_name].rolling(rows_forward).apply(lambda x: list(x).index(max(x))+1))[rows_forward:]
    empty_ = [0]*rows_forward
    rolling_max = rolling_max + empty_
    max_lag = max_lag + empty_
    data['{}_{}'.format(column_name, rows_forward)] = rolling_max
    data['{}_lag_{}'.format(column_name, rows_forward)] = max_lag
    return data

In [48]:
add_rolling_columns('High', 5).head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,High_max,high_5,max_high_lag,High_5,High_lag_5
0,2020-01-02,74.059998,75.150002,73.797501,75.087502,73.561546,135480400,77.607498,77.607498,5.0,77.607498,5.0
1,2020-01-03,74.287498,75.144997,74.125000,74.357498,72.846359,146322800,78.167503,78.167503,5.0,78.167503,5.0
2,2020-01-06,73.447502,74.989998,73.187500,74.949997,73.426834,118387200,79.267502,79.267502,5.0,79.267502,5.0
3,2020-01-07,74.959999,75.224998,74.370003,74.597504,73.081490,108872000,79.392502,79.392502,5.0,79.392502,5.0
4,2020-01-08,74.290001,76.110001,74.290001,75.797501,74.257103,132079200,79.392502,79.392502,4.0,79.392502,4.0
5,2020-01-09,76.809998,77.607498,76.550003,77.407501,75.834381,170108400,79.392502,79.392502,3.0,79.392502,3.0
6,2020-01-10,77.650002,78.167503,77.062500,77.582497,76.005821,140644800,79.684998,79.684998,5.0,79.684998,5.0
7,2020-01-13,77.910004,79.267502,77.787498,79.239998,77.629646,121532000,79.754997,79.754997,5.0,79.754997,5.0
8,2020-01-14,79.175003,79.392502,78.042503,78.169998,76.581383,161954400,79.997498,79.997498,5.0,79.997498,5.0
9,2020-01-15,77.962502,78.875000,77.387497,77.834999,76.253197,121923600,79.997498,79.997498,4.0,79.997498,4.0


In [11]:
max_val = data['High'].shift(5).max()


137.97999572753906

In [10]:
data['High'].shift(-10).max()

138.7899932861328

In [34]:
import numpy as np
def forward_calc(df, column, n, calc_type):

    # check if valid calc type
    if calc_type not in ['min', 'max']:
        raise ValueError('calc_type must be one of min or max')

    # get length of df
    df_len = len(df)
    
    # set up list to store results
    results = []
    
    # loop through each row
    for i in range(df_len):
        # get the index of the nth row
        row_index = i + n
        
        # if nth row is not in df
        if row_index >= df_len:
            # store None in results
            results.append(None)
        else:
            # get column values from nth row
            col_vals = df.loc[i:row_index][column]
            
            # use appropriate calculation
            if calc_type == 'min':
                res = np.min(col_vals)
            else:
                res = np.max(col_vals)
            # store result in results
            results.append(res)
            
    var_ = column + "_" + calc_type
    df[var_] = results
    
    # return list of results
    return df

In [35]:

forward_calc(data,'High',5,'max').tail(12)

,Date,Open,High,Low,Close,Adj Close,Volume,target,High_max
240,2020-12-14,122.599998,123.349998,121.540001,121.779999,120.335449,79184500,1,129.580002
241,2020-12-15,124.339996,127.900002,124.129997,127.879997,126.363091,157243700,1,134.410004
242,2020-12-16,127.410004,128.369995,126.559998,127.809998,126.293922,98208600,0,134.410004
243,2020-12-17,128.899994,129.580002,128.039993,128.699997,127.173363,94359800,0,134.410004
244,2020-12-18,128.960007,129.100006,126.120003,126.660004,125.157578,192541500,1,137.339996
245,2020-12-21,125.019997,128.309998,123.449997,128.229996,126.708931,121251600,1,138.789993
246,2020-12-22,131.610001,134.410004,129.649994,131.880005,130.315643,168904800,0,138.789993
247,2020-12-23,132.160004,132.429993,130.779999,130.960007,129.406555,88223700,0,NaN
248,2020-12-24,131.320007,133.460007,131.100006,131.970001,130.404572,54930100,0,NaN
249,2020-12-28,133.990005,137.339996,133.509995,136.690002,135.068588,124486200,0,NaN


In [26]:
data.reset_index(inplace=True)